In [10]:
# Search to RSS
# Queries a search engine (now Google), scrapes the result and outputs as RSS
from bs4 import BeautifulSoup
import requests

In [11]:
# form and execute query

site = "asiaone.com"
query = "Chia Teck Leng"
query = query.replace(' ', '+')

r = requests.get('https://www.google.com.sg/search?q=site%3A{}+{}'.format(site, query))
soup = BeautifulSoup(r.text, "html.parser")

In [12]:
# extract links
links = []
for item in soup.find_all('h3', attrs={'class' : 'r'}):
    links.append(item.a['href'][7:]) # [7:] strips the /url?q= prefix

print(links)

['http://www.asiaone.com/singapore/guilty-charged-chia-teck-leng-led-double-life-and-cheated-banks-millions&sa=U&ved=0ahUKEwjsxs6YqebZAhXBpY8KHQYgBjEQFggUMAA&usg=AOvVaw1YN3Cb5OiClnI23i0KoMWd', 'http://www.asiaone.com/News/The%252BBusiness%252BTimes/Story/A1Story20090902-165222.html&sa=U&ved=0ahUKEwjsxs6YqebZAhXBpY8KHQYgBjEQFggbMAE&usg=AOvVaw11PugYVTmmXXM3WdM25-wi', '?q=site:asiaone.com+Chia+Teck+Leng&ie=UTF-8&prmd=ivns&tbm=isch&tbo=u&source=univ&sa=X&ved=0ahUKEwjsxs6YqebZAhXBpY8KHQYgBjEQsAQIIQ', 'http://www.asiaone.com/print/News/AsiaOne%252BNews/Singapore/Story/A1Story20071001-27929.html&sa=U&ved=0ahUKEwjsxs6YqebZAhXBpY8KHQYgBjEQFggtMAc&usg=AOvVaw1yLCUE3ngV-K1BgH7hiBcp', 'http://www.asiaone.com/print/News/Latest%252BNews/Singapore/Story/A1Story20121210-388821.html&sa=U&ved=0ahUKEwjsxs6YqebZAhXBpY8KHQYgBjEQFggyMAg&usg=AOvVaw1BFudsubNYfgPh4e7nTgZi', 'http://www.asiaone.com/News/AsiaOne%252BNews/Singapore/Story/A1Story20071003-28296.html&sa=U&ved=0ahUKEwjsxs6YqebZAhXBpY8KHQYgBjEQFgg4MAk&

In [13]:
# crawl links, cleanup responses
from readability.readability import Document
import html2text

rsscontent = []

for link in links:
    print(link.split('&')[0])
    try:
        if "pdf" in link:
            raise Exception('general exceptions not caught by specific handling')
        r = requests.get(link.split('&')[0])
    except:
        print("can't grab: " + link.split('&')[0])
    readable_article = html2text.html2text(Document(r.text).summary())
    readable_title = html2text.html2text(Document(r.text).short_title())
    readable_text = {"title": readable_title,
                     "link": link.split('&')[0],
                    "description": readable_article}
    rsscontent.append(readable_text)

print(rsscontent[0])


http://www.asiaone.com/singapore/guilty-charged-chia-teck-leng-led-double-life-and-cheated-banks-millions
http://www.asiaone.com/News/The%252BBusiness%252BTimes/Story/A1Story20090902-165222.html
?q=site:asiaone.com+Chia+Teck+Leng
can't grab: ?q=site:asiaone.com+Chia+Teck+Leng
http://www.asiaone.com/print/News/AsiaOne%252BNews/Singapore/Story/A1Story20071001-27929.html
http://www.asiaone.com/print/News/Latest%252BNews/Singapore/Story/A1Story20121210-388821.html
http://www.asiaone.com/News/AsiaOne%252BNews/Singapore/Story/A1Story20071003-28296.html
http://www.asiaone.com/News/The%252BBusiness%252BTimes/Story/%252527How%252Bdid%252BChia%252Bmanage%252Bto%252Bkeep%252BAPB%252Bin%252Bthe%252Bdark%25253F%252527.html
http://www.asiaone.com/print/News/The%252BBusiness%252BTimes/Story/Two%252Bbanks%252Bgive%252Bup%252Bclaims%252Bagainst%252BAPB%252Bover%252BChia%252Bswindle.html
http://www.asiaone.com/print/News/The%252BBusiness%252BTimes/Story/Bank%252Bdidn%252527t%252Bpick%252Bup%252Balarm%25

In [14]:
# form the RSS feed

import lxml.etree
import lxml.builder

E = lxml.builder.ElementMaker()

channel = E.channel(
    E.title("Custom RSS Feed: " + site + " " + query), 
    E.link("No Link"),
    E.description("Empty")
)

for article in rsscontent:
    try:
        mytitle = E.title(article["title"])
        mylink = E.link(article["link"])
        mydesc = E.description(article["description"])
        item = E.item(mytitle, mylink, mydesc)
    except ValueError:
        print(repr(article["title"]))
        print(repr(article["link"]))
        print(repr(article["description"]))
        raise
        
    channel.append(item)
    top = page = E.top(channel)


In [15]:
import sys
et = lxml.etree.ElementTree(channel)

file = open("rss.xml", "wb")
et.write(sys.stdout, pretty_print=True)
et.write(file)
file.close()

<channel>
  <title>Custom RSS Feed: asiaone.com Chia+Teck+Leng</title>
  <link>No Link</link>
  <description>Empty</description>
  <item>
    <title>Guilty As Charged: Chia Teck Leng led a double life and cheated banks of
millions

</title>
    <link>http://www.asiaone.com/singapore/guilty-charged-chia-teck-leng-led-double-life-and-cheated-banks-millions</link>
    <description>This story was first published in July 2015 in an e-book titled Guilty As
Charged: 25 Crimes That Have Shaken Singapore Since 1965. A collaboration
between The Straits Times and the Singapore Police Force, the e-book appeared
in The Straits Times Star E-books app. Read the other crime stories here.
(Warning: Some content in these stories may be disturbing for some
individuals.)  
  
**High-rolling hustler (2004)**

 **"Unremarkable" family man, who led double life as the darling of offshore
casinos, cheated banks of millions**

Living in a Serangoon Road condominium with his wife and two teenage sons -
Chia Teck